In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip /content/drive/MyDrive/data.zip -d /content/

Archive:  /content/drive/MyDrive/data.zip
   creating: /content/data/
  inflating: /content/data/swepam_clean.csv  
  inflating: /content/data/floods.csv  
  inflating: /content/data/.DS_Store  
  inflating: /content/__MACOSX/data/._.DS_Store  
  inflating: /content/data/mag_clean.csv  
  inflating: /content/data/floodphenomena.csv  
  inflating: /content/__MACOSX/data/._floodphenomena.csv  
  inflating: /content/data/sfa.csv   
  inflating: /content/data/sis_clean.csv  
  inflating: /content/data/epam_clean.csv  


In [ ]:
! pip install pyts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.4 MB/s 


In [ ]:
import sklearn
import numpy as np
import pandas as pd
import datetime
from pyts.approximation import SymbolicFourierApproximation
from datetime import datetime

# Loading data and adding date column

In [ ]:
epam = pd.read_csv('epam_clean.csv')
epam.drop(axis=1, columns=['Unnamed: 0'], inplace=True)
epam['timestamp'] =  pd.to_datetime(epam.datetime, format='%Y-%m-%d %H:%M:%S')
epam['date'] = epam.timestamp.dt.date

swepam = pd.read_csv('swepam_clean.csv')
swepam.drop(axis=1, columns=['Unnamed: 0'], inplace=True)
swepam['timestamp'] =  pd.to_datetime(swepam.datetime, format='%Y-%m-%d %H:%M:%S')
swepam['date'] = swepam.timestamp.dt.date

mag = pd.read_csv('mag_clean.csv')
mag.drop(axis=1, columns=['Unnamed: 0'], inplace=True)
mag['timestamp'] =  pd.to_datetime(mag.datetime, format='%Y-%m-%d %H:%M:%S')
mag['date'] = mag.timestamp.dt.date

sis = pd.read_csv('sis_clean.csv')
sis.drop(axis=1, columns=['Unnamed: 0'], inplace=True)
sis['timestamp'] =  pd.to_datetime(sis.datetime, format='%Y-%m-%d %H:%M:%S')
sis['date'] = sis.timestamp.dt.date

# Aggregating by date using SFA

### Putting all column values in one list per day

In [ ]:
# Example before transformation
mag.head(2)

,Bx,By,Bz,Bt,Lat.,Long.,datetime,timestamp,date
0,-6.2,-3.1,-0.1,6.9,-0.6,206.5,2001-08-07 00:00:00,2001-08-07 00:00:00,2001-08-07
1,-6.3,-3.3,-0.3,7.1,-2.7,207.8,2001-08-07 00:01:00,2001-08-07 00:01:00,2001-08-07


In [ ]:
mag_wanted_columns = ['Bx', 'By', 'Bz', 'Bt', 'Lat.', 'Long.', 'date']
mag_agg_grouped = mag[mag_wanted_columns].groupby(['date'])
mag_listed = mag_agg_grouped[mag_wanted_columns[:-1]].agg(lambda x : list(x.to_numpy()))
mag_listed.set_index(pd.DatetimeIndex(mag_listed.index), inplace=True)

In [ ]:
# Example after transformation
mag_listed.head(2)

,Bx,By,Bz,Bt,Lat.,Long.
date,,,,,,
2001-08-07,"[-6.2, -6.3, -6.2, -6.1, -6.1, -6.1, -6.5, -6....","[-3.1, -3.3, -3.1, -3.4, -3.5, -3.5, -3.0, -3....","[-0.1, -0.3, -0.6, -0.6, -0.4, 0.0, -0.3, 0.0,...","[6.9, 7.1, 6.9, 7.0, 7.0, 7.0, 7.2, 7.2, 7.0, ...","[-0.6, -2.7, -4.7, -4.8, -3.1, 0.2, -2.3, -0.4...","[206.5, 207.8, 206.5, 209.2, 210.0, 209.8, 204..."
2001-08-08,"[-5.8, -5.3, -5.1, -5.4, -5.1, -5.6, -6.6, -6....","[1.5, 2.3, 2.6, 2.7, 3.0, 3.2, 1.5, 2.5, 3.3, ...","[-2.6, -2.8, -2.7, -1.8, -1.1, 0.6, -0.2, -0.9...","[6.5, 6.4, 6.3, 6.3, 6.1, 6.5, 6.7, 6.7, 6.5, ...","[-23.6, -26.2, -25.1, -16.5, -10.2, 5.1, -1.5,...","[165.8, 156.9, 152.8, 153.6, 149.5, 149.8, 167..."


In [ ]:
swepam_wanted_columns = ['proton_density', 'bulk_speed', 'ion_temperature', 'date']
swepam_agg_grouped = swepam[swepam_wanted_columns].groupby(['date'])
swepam_listed = swepam_agg_grouped[swepam_wanted_columns[:-1]].agg(lambda x : list(x.to_numpy()))
swepam_listed.set_index(pd.DatetimeIndex(swepam_listed.index), inplace=True)

In [ ]:
epam_wanted_columns = ['38-53', '175-315', '47-65', '112-187', '310-580',
                       '761-1220', '060-1910', 'anis_ratio', 'date']
epam_agg_grouped = epam[epam_wanted_columns].groupby(['date'])
epam_listed = epam_agg_grouped[epam_wanted_columns[:-1]].agg(lambda x : list(x.to_numpy()))
epam_listed.set_index(pd.DatetimeIndex(epam_listed.index), inplace=True)

In [ ]:
sis_wanted_columns = ['> 10 MeV', '> 30 MeV', 'date']
sis_agg_grouped = sis[sis_wanted_columns].groupby(['date'])
sis_listed = sis_agg_grouped[sis_wanted_columns[:-1]].agg(lambda x : list(x.to_numpy()))
sis_listed.set_index(pd.DatetimeIndex(sis_listed.index), inplace=True)

# SFA

SFA demands that timeseries that are being transformed have the same number of timestamps.

SFA is done on two df because (sis, epam) are on 5min basis and (mag, swepam) are on a minute basis so they have different number of timestamps per day.

### Adjust length of daily timeseries

If timeseries happen to vary a bit in length due to some previous error, function slick_array will reduce their length to the length of the shortest

In [ ]:
def slick_array(array):
    listic = list(array)
    min_size = min(list(map(len, listic)))
    return np.array(list(map(lambda x: x[:min_size], listic)))

### Making an object required for transformation
Each field in dataframe contains a list that represents a daily timeseries. Each list in one row needs to be turned to numpy.array and all of lists in one row need to be put in one numpy.array. For some reason I couldn't manage to do a more simple job :)

In [ ]:
def list_to_numpy(listici_arg):
    listici = slick_array(listici_arg)
    res = np.array([listici[0]]).reshape(-1, 1)
    for listic in listici[1:]:
        res = np.concatenate((res, np.array([listic]).reshape(-1, 1)), axis = 1)
    return res

### SFA transformation

In [ ]:
def transform(array, num_coefs=3, num_bins=10):
    x = list_to_numpy(array)
    if x.shape[0] < num_bins:
        return []
    if x.shape[1] < num_coefs:
        num_coefs=x.shape[1]
    transformer = SymbolicFourierApproximation(n_coefs=num_coefs, n_bins=num_bins)    
    transformed = transformer.fit_transform(x)
    return transformed.flatten()

In [ ]:
resManji = pd.concat([sis_listed, epam_listed], axis=1, join="inner")
resVeci = pd.concat([mag_listed, swepam_listed], axis=1, join="inner")
# Example before transformation
resManji.head(1)

,> 10 MeV,> 30 MeV,38-53,175-315,47-65,112-187,310-580,761-1220,060-1910,anis_ratio
date,,,,,,,,,,
2001-08-07,"[0.831, 0.82, 0.821, 0.805, 0.809, 0.801, 0.80...","[0.587, 0.574, 0.576, 0.571, 0.574, 0.561, 0.5...","[694.0, 704.0, 797.0, 758.0, 650.0, 712.0, 724...","[11.6, 14.7, 13.4, 16.0, 16.2, 13.1, 14.9, 18....","[1130.0, 1210.0, 1560.0, 1490.0, 1360.0, 1110....","[123.0, 120.0, 131.0, 139.0, 133.0, 117.0, 126...","[6.65, 7.18, 7.67, 8.0, 7.18, 6.49, 6.26, 9.33...","[0.633, 0.946, 0.889, 0.666, 0.903, 0.925, 0.6...","[0.193, 0.252, 0.321, 0.234, 0.19, 0.257, 0.27...","[0.9, 0.68, 1.38, 0.89, 0.43, 0.83, 0.94, 1.02..."


In [ ]:
print("#1 Transforming smaller set...")
resManji_transformed_1 = resManji.apply(transform, axis=1, args=(3,5))
print("#1 Transforming smaller set DONE")

print("#1 Transforming bigger set...")
resVeci_transformed_1 = resVeci.apply(transform, axis=1, args=(3,5))
print("#1 Transforming bigger set DONE")

# Example after transformation
resManji_transformed_1.head()

#1 Transforming smaller set...
#1 Transforming smaller set DONE
#1 Transforming bigger set...
#1 Transforming bigger set DONE


date
2001-08-07    [e, a, a, e, a, a, e, a, a, e, a, a, e, a, a, ...
2001-08-08    [d, b, b, d, a, b, e, b, a, e, a, a, e, c, a, ...
2001-08-09    [b, a, d, b, b, d, a, a, e, a, a, e, c, a, c, ...
2001-08-10    [a, e, b, a, e, b, a, e, b, a, e, b, a, e, b, ...
2001-08-11    [a, e, e, a, e, e, a, e, e, a, e, e, a, e, e, ...
dtype: object

In [ ]:
print("#2 Transforming smaller set...")
resManji_transformed_2 = resManji.apply(transform, axis=1, args=(3,7))
print("#2 Transforming smaller set DONE")

print("#2 Transforming bigger set...")
resVeci_transformed_2 = resVeci.apply(transform, axis=1, args=(3,7))
print("#2 Transforming bigger set DONE")

#2 Transforming smaller set...
#2 Transforming smaller set DONE
#2 Transforming bigger set...
#2 Transforming bigger set DONE


### Joining two dfs
Join both transformed dfs and join their lists into one

In [ ]:
def flatten_row(row):
    return np.concatenate((row[0], row[1]))

In [ ]:
res_1 = pd.concat([resManji_transformed_1, resVeci_transformed_1], axis=1, join="inner").apply(flatten_row, axis=1)
res_1.head(3)

date
2001-08-07    [e, a, a, e, a, a, e, a, a, e, a, a, e, a, a, ...
2001-08-08    [d, b, b, d, a, b, e, b, a, e, a, a, e, c, a, ...
2001-08-09    [b, a, d, b, b, d, a, a, e, a, a, e, c, a, c, ...
dtype: object

In [ ]:
res_2 = pd.concat([resManji_transformed_2, resVeci_transformed_2], axis=1, join="inner").apply(flatten_row, axis=1)
res_2.head(3)

date
2001-08-07    [g, a, a, g, a, a, g, a, a, g, a, a, g, a, a, ...
2001-08-08    [f, b, b, f, b, c, g, c, a, g, b, a, g, d, a, ...
2001-08-09    [c, b, e, b, c, f, a, a, g, a, a, g, d, a, d, ...
dtype: object

# Adding class label

### Loading flood dates

In [ ]:
print("============= Processing floods set..")
floods = pd.read_csv("floods.csv", index_col="date").drop(["Unnamed: 0"], axis =1)
floods.set_index(pd.DatetimeIndex(floods.index), inplace=True)
floods_new = pd.read_csv("floodphenomena.csv")
floods_new = floods_new[(floods_new["StartDate"].notna()) | (floods_new["EndDate"].notna())]
floods_new["StartDateTime"] = pd.to_datetime(floods_new["StartDate"], infer_datetime_format=True)
floods_new["StartDate"] = floods_new["StartDateTime"].dt.date
floods_new["EndDateTime"] = pd.to_datetime(floods_new["EndDate"], infer_datetime_format=True)
floods_new["EndDate"] = floods_new["EndDateTime"].dt.date
floods_new = floods_new[floods_new["StartDate"] >= datetime.strptime("2001-01-01", "%Y-%m-%d").date()]
flood_dates = pd.concat([floods_new["StartDateTime"], floods[floods["flood"]].index.to_series()], axis = 0)
print("============= Processing floods set DONE")

============= Processing floods set..
============= Processing floods set DONE


### Adding class according to flood dates
True  -> date from resulting aggregation is in flood dates

False -> otherwise

Also, join lists into one string

In [ ]:
print("Making df 1 ...")
df_1 = pd.DataFrame(columns=["data", "class"], index=res_1.index)
df_1["data"] = res_1.apply(lambda x: "".join(x))
df_1["class"] = res_1.index.isin(flood_dates)
df_1.head(3)
print("Making df 1 DONE")

Making df 1 ...
Making df 1 DONE


In [ ]:
print("Making df 2 ...")
df_2 = pd.DataFrame(columns=["data", "class"], index=res_2.index)
df_2["data"] = res_2.apply(lambda x: "".join(x))
df_2["class"] = res_2.index.isin(flood_dates)
df_2.head(3)
print("Making df 2 DONE")

Making df 2 ...
Making df 2 DONE


In [ ]:
print("Saving df1 ...")
df_1.to_csv("sfa_1.csv")
print("Saving df1 DONE")

Saving df1 ...
Saving df1 DONE


In [ ]:
print("Saving df2 ...")
df_2.to_csv("sfa_2.csv")
print("Saving df2 DONE")

Saving df2 ...
Saving df2 DONE


In [ ]:
time_period = pd.date_range(start='2001-08-07', end='2015-12-31')

In [ ]:
df_1 = df_1.reset_index()
df_1

,date,data,class
0,2001-08-07,eaaeaaeaaeaaeaaeaaeaaeaaeaaeaaeaaeaaeaaeaaeaae...,False
1,2001-08-08,dbbdabebaeaaecadabcbcdbbcadeaaecadbbedadabdcbb...,False
2,2001-08-09,badbbdaaeaaecaccacbadcacdbbcaccbcbadabeaaebadc...,False
3,2001-08-10,aebaebaebaebaebaecaedaedaedaedaecaecaecaecaeca...,False
4,2001-08-11,aeeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeea...,False
...,...,...,...
7446,2022-04-28,aedbecadeadeaeebddadeaeebedaeeaeeaeebddadeaeeb...,False
7447,2022-04-29,adeaceadeadeadeadeadebddaceaceaddaceadeadeaddb...,False
7448,2022-04-30,cbbcbcbdcbccbccbdccccbccbdcbdcbdcbdcbdcbdcbdcb...,False
7449,2022-05-01,ebaebaebaeaaebaeaaeaaebaebaebaebaecaebaecaecae...,False


In [ ]:
df_1['date'] = df_1['date'].astype('datetime64')
df_1_2015 = df_1[df_1['date'].isin(time_period)]
df_1_2015 = df_1_2015.set_index('date')
df_1_2015

,data,class
date,,
2001-08-07,eaaeaaeaaeaaeaaeaaeaaeaaeaaeaaeaaeaaeaaeaaeaae...,False
2001-08-08,dbbdabebaeaaecadabcbcdbbcadeaaecadbbedadabdcbb...,False
2001-08-09,badbbdaaeaaecaccacbadcacdbbcaccbcbadabeaaebadc...,False
2001-08-10,aebaebaebaebaebaecaedaedaedaedaecaecaecaecaeca...,False
2001-08-11,aeeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeea...,False
...,...,...
2015-12-27,baecbdcadcbdcbcbbddbcabebcddcbcdcbdeacebcdbbdb...,False
2015-12-28,cbccadabebcdabebadbadaaecacbbdacecaccacabebadb...,False
2015-12-29,eeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeeaeeae...,False


In [ ]:
row_num_df1 = df_1_2015.shape[0]
row_num_df1

5156

In [ ]:
import math

In [ ]:
df1_train_row_num = math.floor(row_num_df1*0.8)
df1_valid_row_num = math.floor(row_num_df1*0.1)
df1_test_row_num  = math.floor(row_num_df1*0.1)

In [ ]:
df1_train = df_1_2015.iloc[:df1_train_row_num+1, :]
df1_valid = df_1_2015.iloc[df1_train_row_num+1:df1_train_row_num+df1_valid_row_num+1, :]
df1_test  = df_1_2015.iloc[df1_train_row_num+df1_valid_row_num+1:, :]

In [ ]:
df1_train.to_csv("sfa_1_train.csv") 
df1_valid.to_csv("sfa_1_valid.csv")
df1_test.to_csv("sfa_1_test.csv") 

In [ ]:
df_2 = df_2.reset_index()
df_2

,date,data,class
0,2001-08-07,gaagaagaagaagaagaagaagaagaagaagaagaagaagbafabf...,False
1,2001-08-08,fbbfbcgcagbagdaebcdcdfcbdaefabfebeccfeafbbfebc...,False
2,2001-08-09,cbebcfaagaagdaddadcafdaeeccdbdccdbbfbbfbagcaed...,False
3,2001-08-10,agbagcagcagcagcagdageageagfagfagdagdagcagcagda...,False
4,2001-08-11,aggaggaggaggaggaggaggaggaggaggaggaggaggaggagga...,False
...,...,...,...
7446,2022-04-28,bgfcgdafgafgagfbffaegaggbgeafgafgaggbffafgaggb...,False
7447,2022-04-29,affaegaegafgafgafgaegceeadgaegbefbdgaegaegbefb...,False
7448,2022-04-30,dbcdcdcedcddcddcedcedcddceecfdcfdcedcedcfdcedc...,False
7449,2022-05-01,gcagcagbagbagbagaagabgbbgcagcafcbgdafbbfdbgdaf...,False


In [ ]:
df_2['date'] = df_2['date'].astype('datetime64')
df_2_2015 = df_2[df_2['date'].isin(time_period)]
df_2_2015 = df_2_2015.set_index('date')
df_2_2015

,data,class
date,,
2001-08-07,gaagaagaagaagaagaagaagaagaagaagaagaagaagbafabf...,False
2001-08-08,fbbfbcgcagbagdaebcdcdfcbdaefabfebeccfeafbbfebc...,False
2001-08-09,cbebcfaagaagdaddadcafdaeeccdbdccdbbfbbfbagcaed...,False
2001-08-10,agbagcagcagcagcagdageageagfagfagdagdagcagcagda...,False
2001-08-11,aggaggaggaggaggaggaggaggaggaggaggaggaggaggagga...,False
...,...,...
2015-12-27,cbfdbedbecbfdbeccfebdbcgcdfedcdedbegadgbdfbbfc...,False
2015-12-28,dbecaeacgbcfacgcbecbebbgdadcbebcgdaddadacgcafb...,False
2015-12-29,ggafgbggaggafgafgbfgbfgbfgafgbfgafgbfgbggaggbg...,False


In [ ]:
row_num_df2 = df_2_2015.shape[0]
row_num_df2

5156

In [ ]:
df2_train_row_num = math.floor(row_num_df2*0.8)
df2_valid_row_num = math.floor(row_num_df2*0.1)
df2_test_row_num  = math.floor(row_num_df2*0.1)

In [ ]:
df2_train = df_2_2015.iloc[:df2_train_row_num+1, :]
df2_valid = df_2_2015.iloc[df2_train_row_num+1:df2_train_row_num+df2_valid_row_num+1, :]
df2_test =  df_2_2015.iloc[df2_train_row_num+df2_valid_row_num+1:, :]

In [ ]:
df2_test

,data,class
date,,
2014-08-03,bgfafgafgcceafgfdbdbeceedddaegaggagfadgbagdbdg...,False
2014-08-04,gaafaccafebdgaagaagaagaagabagggaabbgbbggaagaag...,False
2014-08-05,ceebeeddbddcbfedebddcaggaggcgdagfbfeaggcfeaffa...,False
2014-08-06,aggagfaggaggbgfaggaggaggaggbfgagfcfeafgaggbgfa...,False
2014-08-07,edcfcbeddagfaggaggagfaggbffceeeecbffbffdddedca...,False
...,...,...
2015-12-27,cbfdbedbecbfdbeccfebdbcgcdfedcdedbegadgbdfbbfc...,False
2015-12-28,dbecaeacgbcfacgcbecbebbgdadcbebcgdaddadacgcafb...,False
2015-12-29,ggafgbggaggafgafgbfgbfgbfgafgbfgafgbfgbggaggbg...,False


In [ ]:
df2_train.to_csv("sfa_2_train.csv") 
df2_valid.to_csv("sfa_2_valid.csv")
df2_test.to_csv("sfa_2_test.csv")  

In [ ]:
! cp sfa_1_train.csv /content/drive/MyDrive/NI_results/sfa_sets/

In [ ]:
! cp sfa_1_valid.csv /content/drive/MyDrive/NI_results/sfa_sets/

In [ ]:
! cp sfa_1_test.csv /content/drive/MyDrive/NI_results/sfa_sets/

In [ ]:
! cp sfa_2_train.csv /content/drive/MyDrive/NI_results/sfa_sets/

In [ ]:
! cp sfa_2_valid.csv /content/drive/MyDrive/NI_results/sfa_sets/

In [ ]:
! cp sfa_2_test.csv /content/drive/MyDrive/NI_results/sfa_sets/